In [0]:
%pip install --quiet --upgrade scikit-learn==1.4.2 tensorflow==2.16.1 scikeras==0.13.0 joblib

print("Dependências de MLOps instaladas/verificadas.")
print("Versões compatíveis: scikit-learn==1.4.2, tensorflow==2.16.1, scikeras==0.13.0")
print("Reiniciando o kernel Python agora para carregar as novas bibliotecas...")


# Esta função nativa do Databricks reinicia o kernel Python.
# Todas as variáveis serão limpas, o que é o comportamento esperado 
# ao instalar bibliotecas.

In [0]:
%restart_python

In [0]:
# --- Imports ---
import numpy as np
import pandas as pd
import tensorflow as tf
import mlflow
import joblib
import json
import os
import warnings
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import clone
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, recall_score, precision_score, f1_score
from tensorflow.keras.callbacks import EarlyStopping
from mlflow.models.signature import infer_signature
from datetime import datetime

# --- Configurações Globais ---
# A View de features que já existe e será lida
VIEW_NAME = "transacoes_db.feature_store.in_live_features"

# --- MUDANÇA PRINCIPAL AQUI ---
# Diretório em seu Volume do Unity Catalog para salvar os artefatos
# (Assumindo que você queira manter uma subpasta 'pix_fraud' dentro do volume)
MODEL_ARTIFACTS_DIR = "/Volumes/transacoes_db/copper/files/pix_fraud/production_artifacts/"
# --- FIM DA MUDANÇA ---

# Semente para reprodutibilidade
SEED = 42

# --- Configurações de Ambiente ---
warnings.filterwarnings('ignore')

# Garantir que o diretório de artefatos exista no Volume
os.makedirs(MODEL_ARTIFACTS_DIR, exist_ok=True)
print(f"Diretório de artefatos garantido: {MODEL_ARTIFACTS_DIR}")

# --- Sementes de Aleatoriedade ---
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

# Inicializar SparkSession
spark = SparkSession.builder.getOrCreate()

In [0]:
%sql
/* CONTEXTO DA VIEW: transacoes_db.feature_store.in_live_features
(Esta célula %sql é apenas para documentação, a view já existe)

Esta view agrega dados transacionais com perfis e features de janela temporal.
*
SELECT
  -- Colunas da transação
  ft.valor AS valor_transacao,
  ft.data AS data_transacao,
  ft.id_conta_origem AS id_conta_pagador,
  ft.id_conta_destino AS id_conta_recebedor,
  ft.id_tipo_iniciacao_pix AS tipo_iniciacao_pix_id,
  ft.id_finalidade_pix AS finalidade_pix_id,
  
  -- Targets (Rótulos)
  ft.is_fraud AS transacao_fraudulenta, -- (Binário: 0 ou 1)
  ft.fraud_type AS tipo_fraude,         -- (Multiclasse: 'scam', 'roubo', etc.)

  -- Perfis (Pagador e Recebedor)
  conta_orig.saldo AS pagador_saldo,
  conta_orig.aberta_em AS pagador_conta_aberta_em,
  conta_orig.id_tipo_conta AS pagador_tipo_conta_id,
  cliente_orig.id_natureza AS pagador_natureza_id,
  cliente_orig.nascido_em AS pagador_data_nascimento,
  conta_dest.saldo AS recebedor_saldo,
  conta_dest.aberta_em AS recebedor_conta_aberta_em,
  conta_dest.id_tipo_conta AS recebedor_tipo_conta_id,
  cliente_dest.id_natureza AS recebedor_natureza_id,
  cliente_dest.nascido_em AS recebedor_data_nascimento,

  -- Features de Tempo Real (Window Functions)
  ft.pagador_txs_ultimas_24h,
  ft.pagador_valor_ultimas_24h,
  ft.recebedor_txs_ultima_1h,
  ft.recebedor_valor_ultima_1h,
  ft.pagador_segundos_desde_ultima_tx
FROM
  transacoes_db.feature_store.in_live_features AS ft
... (JOINS com tabelas copper de perfis) ...
limit 1

In [0]:
print(f"Carregando dados da view: {VIEW_NAME}...")
# 1. Carregar dados do Delta Lake (Spark)
df_spark = spark.read.table(VIEW_NAME)

# 2. Converter para Pandas
# Assumindo que o dataset cabe na memória do driver para treinamento com Sklearn/Keras
df_pandas = df_spark.toPandas()
print(f"Dados carregados. Total de {len(df_pandas)} registros.")

# 3. Separar Features (X) e Targets (y)
# IDs não são features para o modelo
features_to_drop = ['transacao_fraudulenta', 'tipo_fraude', 'id_conta_pagador', 'id_conta_recebedor']
X = df_pandas.drop(columns=features_to_drop)
y_binary = df_pandas['transacao_fraudulenta']
y_multiclass = df_pandas['tipo_fraude'] # Este ainda é string ('scam', 'legitimo', etc.)

# 4. Divisão em Treino (70%), Validação (15%) e Teste (15%)
# Primeiro, 70% treino e 30% temporário (para val/teste)
X_train, X_temp, y_train_binary, y_temp_binary, y_train_multiclass, y_temp_multiclass = train_test_split(
    X, y_binary, y_multiclass, 
    test_size=0.30, 
    random_state=SEED, 
    stratify=y_binary # Garantir proporção de fraude
)

# Segundo, dividir os 30% temporários em 15% validação e 15% teste (50% de 30% = 15%)
X_val, X_test, y_val_binary, y_test_binary, y_val_multiclass, y_test_multiclass = train_test_split(
    X_temp, y_temp_binary, y_temp_multiclass, 
    test_size=0.50, # 50% do temp (que era 30% do total)
    random_state=SEED, 
    stratify=y_temp_binary # Garantir proporção de fraude
)

# Verificar as proporções
print("--- Divisão dos Dados ---")
print(f"Treino:   {X_train.shape[0]} amostras")
print(f"Validação: {X_val.shape[0]} amostras")
print(f"Teste:    {X_test.shape[0]} amostras")

print("\n--- Proporção de Fraude (Binário) ---")
print(f"Treino:    {y_train_binary.value_counts(normalize=True)[1]:.4f}")
print(f"Validação: {y_val_binary.value_counts(normalize=True)[1]:.4f}")
print(f"Teste:     {y_test_binary.value_counts(normalize=True)[1]:.4f}")

In [0]:
# --- Funções Customizadas para Engenharia de Features de Data/Hora ---

def feature_engineer_datetimes(df_in: pd.DataFrame) -> pd.DataFrame:
    """
    Recebe um DataFrame com as colunas de data/hora e retorna
    um DataFrame com features numéricas de engenharia.
    """
    # Criar cópia para evitar SettingWithCopyWarning
    df = df_in.copy()
    
    # DataFrame de saída
    df_out = pd.DataFrame(index=df.index)
    
    # Timestamp atual para cálculos de idade
    now = datetime.now()

    # 1. 'data_transacao'
    dt_tx = pd.to_datetime(df['data_transacao'])
    df_out['tx_hora_do_dia'] = dt_tx.dt.hour
    df_out['tx_dia_da_semana'] = dt_tx.dt.dayofweek
    df_out['tx_mes'] = dt_tx.dt.month

    # 2. Idade das Contas (em dias)
    df_out['pagador_idade_conta_dias'] = (now - pd.to_datetime(df['pagador_conta_aberta_em'])).dt.days
    df_out['recebedor_idade_conta_dias'] = (now - pd.to_datetime(df['recebedor_conta_aberta_em'])).dt.days
    
    # 3. Idade dos Clientes (em anos)
    df_out['pagador_idade_anos'] = ((now - pd.to_datetime(df['pagador_data_nascimento'])).dt.days / 365.25).astype(int)
    df_out['recebedor_idade_anos'] = ((now - pd.to_datetime(df['recebedor_data_nascimento'])).dt.days / 365.25).astype(int)

    # Lidar com possíveis nulos que podem ter sido gerados (ex: datas inválidas)
    df_out = df_out.fillna(0)
    
    return df_out

# --- Definição das Listas de Colunas ---

# Colunas numéricas que entram direto no scaler
numeric_features = [
    'valor_transacao', 
    'pagador_saldo', 
    'recebedor_saldo',
    'pagador_txs_ultimas_24h',
    'pagador_valor_ultimas_24h',
    'recebedor_txs_ultima_1h',
    'recebedor_valor_ultima_1h',
    'pagador_segundos_desde_ultima_tx'
]

# Colunas categóricas para One-Hot Encoding
categorical_features = [
    'tipo_iniciacao_pix_id', 
    'finalidade_pix_id', 
    'pagador_tipo_conta_id', 
    'pagador_natureza_id',
    'recebedor_tipo_conta_id', 
    'recebedor_natureza_id'
]

# Colunas de data/hora para o transformador customizado
datetime_features = [
    'data_transacao', 
    'pagador_conta_aberta_em', 
    'pagador_data_nascimento',
    'recebedor_conta_aberta_em', 
    'recebedor_data_nascimento'
]

# --- Criação dos Pipelines de Transformação ---

# Pipeline para features numéricas
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Pipeline para features categóricas
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Pipeline para features de data/hora
datetime_transformer = Pipeline(steps=[
    ('feature_eng', FunctionTransformer(feature_engineer_datetimes)),
    ('scaler', StandardScaler()) # Escalonar as features de data/hora criadas
])

# --- Montagem do ColumnTransformer (Pré-processador) ---

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('date', datetime_transformer, datetime_features)
    ],
    remainder='drop' # Ignorar colunas não listadas (ex: IDs)
)

print("Pipeline de pré-processamento 'preprocessor' definido com sucesso.")

In [0]:
# --- Função Factory para Modelo Binário ---
def build_binary_model(meta):
    """Constrói o modelo binário para o KerasClassifier."""
    # scikeras injeta n_features_in_ automaticamente
    n_features_in = meta["n_features_in_"]
    
    # Garantir que a semente do TF seja definida dentro da função
    # para reprodutibilidade quando o KerasClassifier for clonado
    tf.random.set_seed(SEED) 

    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(n_features_in,)),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1, activation='sigmoid') # Saída sigmoide para binário
    ])

    model.compile(
        loss='binary_crossentropy', 
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
        metrics=['AUC', tf.keras.metrics.Recall(name='recall'), tf.keras.metrics.Precision(name='precision')]
    )
    return model

# --- Função Factory para Modelo Multiclasse ---
def build_multiclass_model(meta):
    """Constrói o modelo multiclasse para o KerasClassifier."""
    n_features_in = meta["n_features_in_"]
    # scikeras injeta o número de classes únicas (ex: 3 tipos de fraude)
    n_classes_out = meta["n_classes_"]
    
    tf.random.set_seed(SEED)

    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(n_features_in,)),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(n_classes_out, activation='softmax') # Saída softmax
    ])

    model.compile(
        loss='sparse_categorical_crossentropy', # Usar sparse pois os labels são inteiros (0, 1, 2)
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
        metrics=['accuracy']
    )
    return model

print("Funções de construção de modelo Keras ('build_binary_model', 'build_multiclass_model') definidas.")

In [0]:
# Definir callback de Early Stopping
# Parar o treino se a perda na validação (val_loss) não melhorar após 5 épocas
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=5, 
    restore_best_weights=True,
    verbose=1
)

# 1. Iniciar experimento MLflow
mlflow.set_experiment(f"/Users/{dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()}/Pix_Fraud_Detection")

with mlflow.start_run(run_name="Fraud_Binary_Classifier") as run_binary:
    print(f"Iniciando Run MLflow: {run_binary.info.run_id}")
    
    # --- CORREÇÃO AQUI ---
    # O 'scikeras.wrappers' moderno usa o argumento 'model' e não 'build_fn'
    keras_binary_model = KerasClassifier(
        model=build_binary_model,  # <-- MUDANÇA DE 'build_fn' PARA 'model'
        epochs=50, 
        batch_size=256, 
        verbose=1,
        random_state=SEED
    )
    # --- FIM DA CORREÇÃO ---

    # 3. Criar o Pipeline completo
    pipeline_binary = Pipeline([
        ('preprocessor', preprocessor), 
        ('model', keras_binary_model)
    ])

    # 4. Pré-processar dados de validação para Early Stopping
    # O pipeline.fit não transforma automaticamente os dados passados para 'model__validation_data'
    print("Pré-ajustando o processador para transformar dados de validação...")
    # Usamos clone() para não "sujar" o preprocessor do pipeline antes do .fit()
    preprocessor_for_val = clone(preprocessor).fit(X_train)
    X_val_processed = preprocessor_for_val.transform(X_val)
    print(f"Dimensões dos dados de validação processados: {X_val_processed.shape}")
    
    # 5. Treinar o pipeline
    print("Iniciando treinamento do pipeline binário...")
    pipeline_binary.fit(
        X_train, y_train_binary, 
        model__validation_data=(X_val_processed, y_val_binary), 
        model__callbacks=[early_stopping]
    )
    print("Treinamento binário concluído.")

    # 6. Avaliar no set de teste
    y_pred_binary_proba = pipeline_binary.predict_proba(X_test)[:, 1]
    y_pred_binary = (y_pred_binary_proba >= 0.5).astype(int)

    # 7. Logar métricas (Foco em Recall)
    recall = recall_score(y_test_binary, y_pred_binary)
    precision = precision_score(y_test_binary, y_pred_binary)
    f1 = f1_score(y_test_binary, y_pred_binary)
    roc_auc = roc_auc_score(y_test_binary, y_pred_binary_proba)

    mlflow.log_metric("test_recall", recall)
    mlflow.log_metric("test_precision", precision)
    mlflow.log_metric("test_f1", f1)
    mlflow.log_metric("test_roc_auc", roc_auc)
    
    print("\n--- Métricas de Teste (Binário) ---")
    print(f"Recall:    {recall:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"F1-Score:  {f1:.4f}")
    print(f"ROC AUC:   {roc_auc:.4f}")
    print(classification_report(y_test_binary, y_pred_binary))

    # 8. Salvamento Crítico do Artefato (usará o caminho do Volume)
    binary_pipeline_path = os.path.join(MODEL_ARTIFACTS_DIR, "fraud_binary_pipeline.pkl")
    joblib.dump(pipeline_binary, binary_pipeline_path)
    print(f"Pipeline binário salvo em: {binary_pipeline_path}")

    # 9. Logar no MLflow (modelo e artefato)
    signature = infer_signature(X_train, pipeline_binary.predict(X_train))
    mlflow.sklearn.log_model(
        pipeline_binary, 
        "binary_model", 
        signature=signature,
        input_example=X_train.head(5).to_dict(orient='records')
    )
    mlflow.log_artifact(binary_pipeline_path)

print("Célula 6 concluída.")

In [0]:
print("Iniciando preparação para o Modelo 2 (Multiclasse)...")

# 1. Filtrar dados para incluir apenas transações fraudulentas
train_fraud_mask = (y_train_binary == 1)
val_fraud_mask = (y_val_binary == 1)
test_fraud_mask = (y_test_binary == 1)

X_train_fraud = X_train[train_fraud_mask]
y_train_multiclass_fraud = y_train_multiclass[train_fraud_mask]

X_val_fraud = X_val[val_fraud_mask]
y_val_multiclass_fraud = y_val_multiclass[val_fraud_mask]

X_test_fraud = X_test[test_fraud_mask]
y_test_multiclass_fraud = y_test_multiclass[test_fraud_mask]

print(f"Total de amostras de fraude para treino: {len(X_train_fraud)}")
print(f"Total de amostras de fraude para validação: {len(X_val_fraud)}")
print(f"Total de amostras de fraude para teste: {len(X_test_fraud)}")

# 2. Usar LabelEncoder para converter labels string em inteiros
label_encoder = LabelEncoder()
y_train_multiclass_encoded = label_encoder.fit_transform(y_train_multiclass_fraud)
y_val_multiclass_encoded = label_encoder.transform(y_val_multiclass_fraud)
y_test_multiclass_encoded = label_encoder.transform(y_test_multiclass_fraud)

print(f"Classes de fraude encontradas: {label_encoder.classes_}")

# 3. Salvamento Crítico (Mapeamento de Labels) (usará o caminho do Volume)
# Criar mapa {0: 'scam', 1: 'roubo', 2: 'lavagem'}
label_map = {i: str(cls) for i, cls in enumerate(label_encoder.classes_)}
label_map_path = os.path.join(MODEL_ARTIFACTS_DIR, "fraud_type_label_map.json")

with open(label_map_path, 'w') as f:
    json.dump(label_map, f, indent=4)
print(f"Mapa de labels salvo em: {label_map_path}")

# --- Treinamento do Modelo Multiclasse ---

# Callback para o modelo multiclasse
early_stopping_multi = EarlyStopping(
    monitor='val_loss', 
    patience=5, 
    restore_best_weights=True,
    verbose=1
)

with mlflow.start_run(run_name="Fraud_Type_Classifier") as run_multiclass:
    print(f"Iniciando Run MLflow: {run_multiclass.info.run_id}")
    
    # --- CORREÇÃO AQUI ---
    # O 'scikeras.wrappers' moderno usa o argumento 'model' e não 'build_fn'
    keras_multiclass_model = KerasClassifier(
        model=build_multiclass_model, # <-- MUDANÇA DE 'build_fn' PARA 'model'
        epochs=50, 
        batch_size=128, 
        verbose=1,
        random_state=SEED
    )
    # --- FIM DA CORREÇÃO ---

    # 5. Criar o Pipeline completo
    pipeline_multiclass = Pipeline([
        ('preprocessor', preprocessor), 
        ('model', keras_multiclass_model)
    ])
    
    # 6. Pré-processar dados de validação (apenas de fraude)
    print("Pré-ajustando o processador (multiclasse) nos dados de treino de fraude...")
    # Usamos clone() para não "sujar" o preprocessor do pipeline antes do .fit()
    preprocessor_multi_for_val = clone(preprocessor).fit(X_train_fraud)
    X_val_fraud_processed = preprocessor_multi_for_val.transform(X_val_fraud)
    print(f"Dimensões dos dados de validação (fraude) processados: {X_val_fraud_processed.shape}")

    # 7. Treinar o pipeline (APENAS com dados de fraude)
    print("Iniciando treinamento do pipeline multiclasse...")
    pipeline_multiclass.fit(
        X_train_fraud, y_train_multiclass_encoded, 
        model__validation_data=(X_val_fraud_processed, y_val_multiclass_encoded), 
        model__callbacks=[early_stopping_multi]
    )
    print("Treinamento multiclasse concluído.")

    # 8. Avaliar no set de teste (filtrado)
    y_pred_multiclass_encoded = pipeline_multiclass.predict(X_test_fraud)
    
    print("\n--- Métricas de Teste (Multiclasse) ---")
    report_text = classification_report(
        y_test_multiclass_encoded, 
        y_pred_multiclass_encoded, 
        target_names=label_encoder.classes_
    )
    print(report_text)
    
    # Logar métricas
    mlflow.log_text(report_text, "classification_report.txt")
    f1_micro = f1_score(y_test_multiclass_encoded, y_pred_multiclass_encoded, average='micro')
    mlflow.log_metric("test_f1_micro", f1_micro)

    # 9. Salvamento Crítico (usará o caminho do Volume)
    multiclass_pipeline_path = os.path.join(MODEL_ARTIFACTS_DIR, "fraud_type_pipeline.pkl")
    joblib.dump(pipeline_multiclass, multiclass_pipeline_path)
    print(f"Pipeline multiclasse salvo em: {multiclass_pipeline_path}")

    # 10. Logar no MLflow
    signature_multi = infer_signature(X_train_fraud, pipeline_multiclass.predict(X_train_fraud))
    mlflow.sklearn.log_model(
        pipeline_multiclass, 
        "multiclass_model", 
        signature=signature_multi,
        input_example=X_train_fraud.head(5).to_dict(orient='records')
    )
    mlflow.log_artifact(multiclass_pipeline_path)
    mlflow.log_artifact(label_map_path) # Importante logar o mapa também

print("Célula 7 concluída.")

In [0]:
print("--- Iniciando Avaliação Final (Pós-Treino) com Artefatos Salvos ---")

# 1. Carregar artefatos do Volume
try:
    loaded_pipeline_binary = joblib.load(os.path.join(MODEL_ARTIFACTS_DIR, "fraud_binary_pipeline.pkl"))
    loaded_pipeline_multiclass = joblib.load(os.path.join(MODEL_ARTIFACTS_DIR, "fraud_type_pipeline.pkl"))
    
    map_path = os.path.join(MODEL_ARTIFACTS_DIR, "fraud_type_label_map.json")
    with open(map_path, 'r') as f:
        # Converter chaves JSON (string) de volta para inteiros
        loaded_label_map_str = json.load(f)
        loaded_label_map = {int(k): v for k, v in loaded_label_map_str.items()}
        
    print("Artefatos (2 pipelines, 1 mapa) carregados com sucesso do Volume.")

except Exception as e:
    print(f"Erro fatal ao carregar artefatos do Volume: {e}")
    # Se falhar aqui, não podemos continuar
    raise e

# --- Avaliação Modelo 1: Classificação Binária ---
print("\n--- Avaliação Modelo 1: Classificação Binária (em X_test) ---")

# Usar os dados de teste reais (y_test_binary)
y_pred_binary_loaded = loaded_pipeline_binary.predict(X_test)

print(classification_report(y_test_binary, y_pred_binary_loaded, target_names=['Legitimo', 'Fraude']))
print("Matriz de Confusão (Binário):")
print(confusion_matrix(y_test_binary, y_pred_binary_loaded))


# --- Avaliação Modelo 2: Classificação Multiclasse ---
print("\n--- Avaliação Modelo 2: Classificação Multiclasse (em X_test ONDE y_test==1) ---")

# Filtrar o X_test para incluir apenas as fraudes VERDADEIRAS
# (para avaliar o quão bem o 2º modelo classifica os tipos de fraude reais)
X_test_true_fraud = X_test[y_test_binary == 1]

# --- CORREÇÃO AQUI ---
# Usamos 'y_test_multiclass' (o split de teste)
# Em vez de 'y_multiclass' (o dataset completo)
y_test_true_fraud_labels = y_test_multiclass[y_test_binary == 1]
# --- FIM DA CORREÇÃO ---


if not X_test_true_fraud.empty:
    # Prever os índices (0, 1, 2)
    y_pred_multiclass_idx = loaded_pipeline_multiclass.predict(X_test_true_fraud)
    
    # Mapear os índices de volta para os labels string ('scam', 'roubo')
    y_pred_multiclass_labels = [loaded_label_map.get(idx, 'unknown') for idx in y_pred_multiclass_idx]
    
    # Classes reais para o relatório
    true_labels_list = list(loaded_label_map.values())
    
    print(classification_report(y_test_true_fraud_labels, y_pred_multiclass_labels, labels=true_labels_list))
    print("Matriz de Confusão (Multiclasse):")
    print(confusion_matrix(y_test_true_fraud_labels, y_pred_multiclass_labels, labels=true_labels_list))
else:
    print("Não foram encontradas fraudes verdadeiras no conjunto de teste para avaliar o modelo multiclasse.")

print("\nCélula 8 concluída. Avaliação final completa.")

In [0]:
# Esta célula é 100% AUTÔNOMA e só depende dos artefatos salvos na Célula 6 e 7.
# NENHUMA variável ou função das células anteriores é usada here.

import pandas as pd
import joblib
import numpy as np
import json
import os
import warnings
from datetime import datetime

# --- Constantes de Produção ---
# Este deve ser o mesmo diretório do Volume usado no treinamento
MODEL_DIR = "/Volumes/transacoes_db/copper/files/pix_fraud/production_artifacts/"
BINARY_THRESHOLD = 0.5 # Threshold de decisão para fraude binária

# Suprimir warnings de serialização/versão que podem ocorrer
warnings.filterwarnings('ignore')

# --- Carregamento de Artefatos ---

def carregar_artefatos(model_dir: str):
    """
    Carrega todos os artefatos necessários (pipelines e mapa de labels)
    do diretório especificado (no Volume).
    """
    print(f"Carregando artefatos de: {model_dir}")
    try:
        pipeline_binario = joblib.load(os.path.join(model_dir, "fraud_binary_pipeline.pkl"))
        pipeline_multiclass = joblib.load(os.path.join(model_dir, "fraud_type_pipeline.pkl"))

        label_map_path = os.path.join(model_dir, "fraud_type_label_map.json")
        with open(label_map_path, 'r') as f:
            # Converter chaves JSON (string) de volta para inteiros
            label_map_str_keys = json.load(f)
            label_map = {int(k): v for k, v in label_map_str_keys.items()}
            
        print("Artefatos (2 pipelines, 1 mapa de labels) carregados com sucesso.")
        return pipeline_binario, pipeline_multiclass, label_map

    except FileNotFoundError as e:
        print(f"Erro crítico: Arquivo de modelo/mapa não encontrado. {e}")
        return None, None, None
    except Exception as e:
        print(f"Erro inesperado ao carregar artefatos: {e}")
        return None, None, None

# --- Função de Predição em Cascata ---

def prever_fraude(df_novo: pd.DataFrame, pipeline_binario, pipeline_multiclass, label_map) -> pd.DataFrame:
    """
    Executa a predição em cascata para novas transações.
    Recebe um DataFrame Pandas com o schema exato da view de features.
    """
    if pipeline_binario is None or pipeline_multiclass is None or label_map is None:
        raise ValueError("Modelos ou mapa de labels não foram carregados corretamente.")

    if not isinstance(df_novo, pd.DataFrame):
        df_novo = pd.DataFrame([df_novo])
        
    df_results = df_novo.copy()

    # 1. Modelo Binário
    try:
        fraud_probabilities = pipeline_binario.predict_proba(df_novo)[:, 1]
    except Exception as e:
        print(f"Erro na predição binária: {e}")
        df_results['predicted_fraud'] = np.nan
        df_results['fraud_probability'] = np.nan
        df_results['predicted_fraud_type'] = None
        df_results['predicted_type_probability'] = np.nan
        df_results['type_probabilities'] = None
        df_results['resumo_da_predicao'] = "Erro na predição" # <-- NOVO
        return df_results

    df_results['fraud_probability'] = fraud_probabilities
    df_results['predicted_fraud'] = (fraud_probabilities >= BINARY_THRESHOLD).astype(int)

    # Inicializar colunas
    df_results['predicted_fraud_type'] = None
    df_results['predicted_type_probability'] = np.nan
    df_results['type_probabilities'] = [{} for _ in range(len(df_results))]

    # 2. Modelo Multiclasse (só para fraudes)
    indices_fraude = df_results[df_results['predicted_fraud'] == 1].index

    if not indices_fraude.empty:
        df_fraude = df_results.loc[indices_fraude]
        
        try:
            type_probs_matrix = pipeline_multiclass.predict_proba(df_fraude)
            type_predictions_indices = np.argmax(type_probs_matrix, axis=1)
            predicted_type_probs = np.max(type_probs_matrix, axis=1)
            predicted_types = [label_map.get(idx, 'unknown') for idx in type_predictions_indices]
            
            df_results.loc[indices_fraude, 'predicted_fraud_type'] = predicted_types
            df_results.loc[indices_fraude, 'predicted_type_probability'] = predicted_type_probs
            
            type_probs_list = []
            for probs_row in type_probs_matrix:
                row_dict = {label_map.get(i, 'unknown'): float(prob) for i, prob in enumerate(probs_row)}
                type_probs_list.append(row_dict)
            
            df_results.loc[indices_fraude, 'type_probabilities'] = pd.Series(
                type_probs_list, 
                index=indices_fraude, 
                dtype='object'
            )
        except Exception as e:
            print(f"Erro na predição multiclasse: {e}")
            # Se falhar, o tipo de fraude ficará como 'None'

    # --- NOVO: LÓGICA DA STRING DE SAÍDA ---
    
    # 1. Definir o padrão (Não é fraude)
    df_results['resumo_da_predicao'] = "Não é uma fraude."
    
    # 2. Atualizar apenas as linhas que SÃO fraude (usando os índices)
    if not indices_fraude.empty:
        # Pega os tipos de fraude previstos (ex: 'scam', 'roubo', ou 'None' se a predição multiclasse falhou)
        tipos_de_fraude = df_results.loc[indices_fraude, 'predicted_fraud_type']
        
        # Cria a string formatada
        df_results.loc[indices_fraude, 'resumo_da_predicao'] = "É UMA FRAUDE do tipo " + tipos_de_fraude.astype(str)
        
        # Correção para o caso de a predição multiclasse ter falhado e o tipo ser 'None'
        df_results['resumo_da_predicao'] = df_results['resumo_da_predicao'].str.replace("do tipo None", "do tipo desconhecido")

    # --- FIM DA NOVA LÓGICA ---

    return df_results

# --- Exemplo de Uso (Simulação) ---
print("\n--- SIMULANDO EXECUÇÃO DE INFERÊNCIA AUTÔNOMA ---")

# 1. Carregar os artefatos UMA VEZ
p_bin, p_multi, l_map = carregar_artefatos(MODEL_DIR)

if p_bin and p_multi and l_map:
    # 2. Criar um payload de exemplo (usando objetos datetime nativos)
    data_exemplo = [
        {
            # Transação 1: Baixo valor, conta antiga (provavelmente legítima)
            'valor_transacao': 50.20,
            'data_transacao': datetime.now(),
            'tipo_iniciacao_pix_id': 1, 'finalidade_pix_id': 1,
            'pagador_saldo': 1500.00, 'pagador_conta_aberta_em': '2018-05-10T10:00:00',
            'pagador_tipo_conta_id': 1, 'pagador_natureza_id': 1, 'pagador_data_nascimento': '1985-01-15T00:00:00',
            'recebedor_saldo': 3000.00, 'recebedor_conta_aberta_em': '2019-11-20T14:30:00',
            'recebedor_tipo_conta_id': 1, 'recebedor_natureza_id': 1, 'recebedor_data_nascimento': '1990-03-22T00:00:00',
            'pagador_txs_ultimas_24h': 2, 'pagador_valor_ultimas_24h': 150.0,
            'recebedor_txs_ultima_1h': 0, 'recebedor_valor_ultima_1h': 0.0,
            'pagador_segundos_desde_ultima_tx': 86400
        },
        {
            # Transação 2: Alto valor, conta nova, de madrugada (suspeita)
            'valor_transacao': 4000.00,
            'data_transacao': datetime.now().replace(hour=3, minute=30),
            'tipo_iniciacao_pix_id': 2, 'finalidade_pix_id': 2,
            'pagador_saldo': 4600.00, 
            'pagador_conta_aberta_em': datetime.now().replace(day=1),
            'pagador_tipo_conta_id': 1, 'pagador_natureza_id': 1, 'pagador_data_nascimento': '1998-07-10T00:00:00',
            'recebedor_saldo': 0.00, 
            'recebedor_conta_aberta_em': datetime.now().replace(day=2),
            'recebedor_tipo_conta_id': 2, 'recebedor_natureza_id': 2, 'recebedor_data_nascimento': '1999-12-01T00:00:00',
            'pagador_txs_ultimas_24h': 1, 'pagador_valor_ultimas_24h': 0,
            'recebedor_txs_ultima_1h': 5, 'recebedor_valor_ultima_1h': 0,
            'pagador_segundos_desde_ultima_tx': 300
        }
    ]
    
    payload_exemplo = pd.DataFrame(data_exemplo)
    
    # 3. Executar predição
    print("\nExecutando predição no payload de exemplo...")
    resultados = prever_fraude(payload_exemplo, p_bin, p_multi, l_map)
    
    print("\nResultados da Predição (Saída):")
    
    # --- MODIFICADO: Exibir a nova string de resumo ---
    output_columns = ['resumo_da_predicao', 'fraud_probability', 'predicted_type_probability']
    
    # Configurar pandas para exibir floats com 4 casas decimais
    pd.set_option('display.float_format', lambda x: '%.4f' % x)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000) # Deixa a tabela mais larga
    
    print(resultados[output_columns])
else:
    print("Simulação de inferência falhou: Artefatos não puderam ser carregados.")